Este projeto consiste no Lab 2 do capitulo de Processamento de Linguagem Natural do curso de Machine Learning da Data Science Academy.

## Lab - Collocations e Processamento de Comentários de Avaliações de Hotéis

Collocations são duas ou mais palavras que tendem a aparecer frequentemente juntas, como "Estados Unidos", "Rio Grande do Sul" ou "Machine Learning". Essas palavras podem gerar diversas combinações e por isso o contexto também é importante no processamento de linguagem natural.

Os dois tipos mais comuns de Collocations são bigramas e trigramas. Bigramas são duas palavras adjacentes, como "tomografia computadorizada" ou "mídia social". Trigramas são três palavras adjacentes, como "fora do negócio" ou "Proctor and Gamble".

- Bigramas: (Nome, Nome), (Adjetivo, Nome)
- Trigramas: (Adjetivo/Nome, Qualquer_Item, Adjetivo/Nome)

Mas se escolhermos palavras adjacentes como bigrama ou trigramas, não obteremos frases significativas. Por exemplo, a frase 'Ele usa mídias sociais' contém bigramas: 'Ele usa', 'usa mídias', 'mídias sociais'. "Ele usa" e "usa mídias" não significa nada, enquanto "mídias sociais" é um bigrama significativo.

### Definição do Problema

Dado um conjunto de texto de avaliações (comentários) de hotéis, vamos buscar as Collocations mais relevantes que ajudam a explicar as avaliações!

### Tratamento dos Dados

In [1]:
# Imports
import pandas as pd # versão 2.0.2
import nltk # versão 3.6.1
import spacy # versão 3.7.2
import re # versão 2.2.1
import string
from nltk.corpus import stopwords

In [2]:
# Carregando dados de avaliações de hotéis
# Fonte dos dados: https://datafiniti.co/products/business-data/
avaliacoes_hoteis = pd.read_csv('https://raw.githubusercontent.com/dsacademybr/Datasets/master/dataset7.csv')

In [3]:
# a tabela apresenta diferentes dados de avialiações de hoteis pelo mundo
# para este projeto o que intereça é somente a coluna reviews.text que contem as avaliações comentadas
print(avaliacoes_hoteis.shape)
avaliacoes_hoteis.head(5)

(10000, 25)


,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,...,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,...,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


In [4]:
# extraindo avaliações
comentarios = avaliacoes_hoteis['reviews.text']
# converter para o tipo string
comentarios = comentarios.astype('str')

In [5]:
# função para remover caracteres no ascii
def removeNoAscii(s):
    return "".join(i for i in s if ord(i)<128)

In [6]:
# removendo caracteres no ascii
comentarios = comentarios.map(lambda x: removeNoAscii(x))

### Detectar Idioma Predominante

In [7]:
# para detectar o idioma predominante usaremos as stopwords de todos idioma 
# obtendo stopwords em todos os idiomas
dicionario_stopwords = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
print("Idiomas Disponíveis:")
print(list(dicionario_stopwords.keys()))

Idiomas Disponíveis:
['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [8]:
def idiomas_serie(serie):
    # score de cada idioma
    i_score = [0 for i in dicionario_stopwords.keys()]
    for i in range(serie.shape[0]):
        # tokenizar as palavras
        palavras = set(nltk.wordpunct_tokenize(serie.iloc[i].lower()))
        # buscar o idioma com maior quantidade de stopwords em no conjunto de palavras
        # assume-se que este idioma será o idioma da frase
        lang = max(((lang, len(palavras & stopwords)) for lang, stopwords in dicionario_stopwords.items()),
                   key = lambda x: x[1])[0]
        i_score[list(dicionario_stopwords.keys()).index(lang)]+=1
    # exibir scores
    for i in range(len(i_score)):
        if i_score[i]!=0:
            print(f"{list(dicionario_stopwords.keys())[i]}: {i_score[i]}")
idiomas_serie(comentarios)

arabic: 57
azerbaijani: 6
catalan: 5
danish: 7
dutch: 6
english: 468
finnish: 1
french: 13
german: 9
hinglish: 9397
italian: 5
portuguese: 4
slovene: 8
spanish: 14


In [9]:
# hinglish é uma variante da inglês com o hindi
# para este projeto usaremos english e hinglish como equivalente e portanto a lingua predomintante é o inglês

# função para filtrar os comentários em inglês
def is_englesh(text):
    # aplicar tokenização considerando pontuação
    palavras = set(nltk.wordpunct_tokenize(text.lower()))
    # contar o total de palavras tokenizadas considerando o dicionário de stopwords
    lang = max(((lang, len(palavras & stopwords)) for lang, stopwords in dicionario_stopwords.items()), key = lambda x: x[1])[0]
    # verificar o idioma
    if lang == "english" or lang=="hinglish":
        return True
    else:
        return False

In [10]:
# filtrar os comentários em inglês
comentarios_eg = comentarios[comentarios.apply(is_englesh)]
print("Número de Comentários c/duplicatas:",comentarios_eg.shape[0])
comentarios_eg.drop_duplicates(inplace=True)
print("Número de Comentários s/duplicatas:",comentarios_eg.shape[0])
comentarios_eg.head()

Número de Comentários c/duplicatas: 9865
Número de Comentários s/duplicatas: 9854


0    Our experience at Rancho Valencia was absolute...
1    Amazing place. Everyone was extremely warm and...
2    We booked a 3 night stay at Rancho Valencia to...
3    Currently in bed writing this for the past hr ...
4    I live in Md and the Aloft is my Home away fro...
Name: reviews.text, dtype: object

In [11]:
# carrando modelo para o processamento de linguagem natural em inglês
# en_core_web_sm = English Core Web Small 
# modelo pequeno e leve que é treinado para tarefas básicas de processamento de linguagem natural em inglês
# caso não tenha instalado, executar esta linha: !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [12]:
# função para limpar comentários
def limpa_comentarios(text):
    # remover pontuação
    regex = re.compile('['+re.escape(string.punctuation)+'\\r\\t\\n]')
    # substituir pontuação por espaços em branco usando expressão regular
    nopunct = regex.sub(" ",str(text))
    # lematizar - retornar as palavras conjugadas ou derivadas para sua forma base
    # desativar parser(componentes de análise sintática) e ner(reconhecimento de entidade nomeada)
    doc = nlp(nopunct, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma

In [13]:
comentarios_eg_lemmatized = comentarios_eg.map(limpa_comentarios)

In [14]:
# colocar comentários em minúsculo
comentarios_eg_lemmatized = comentarios_eg_lemmatized.map(lambda x: [word.lower() for word in x])

In [15]:
# palavras agrupadas por comentários
comentarios_eg_lemmatized.head()

0    [our, experience, at, rancho, valencia, be, ab...
1    [amazing, place,  , everyone, be, extremely, w...
2    [we, book, a, 3, night, stay, at, rancho, vale...
3    [currently, in, bed, write, this, for, the, pa...
4    [i, live, in, md, and, the, aloft, be, my, hom...
Name: reviews.text, dtype: object

In [16]:
# unindo todos os tokens em uma lista
comentarios_tokens = [item for items in comentarios_eg_lemmatized
                     for item in items]

### Estratégias para encontrar os collocations mais relevantes

Diferentes métricas podem ser utilizadas para calcular os ngramas mais relevantes.
Neste projeto vamos testar 4 estrátegias e comparar seus resultados 

In [17]:
# métrica de associação de bigramas e trigramas
bigramas = nltk.collocations.BigramAssocMeasures()
trigramas = nltk.collocations.TrigramAssocMeasures()

In [18]:
# buscar bigramas e trigramas nos comentários
buscarBigramas = nltk.collocations.BigramCollocationFinder.from_words(comentarios_tokens)
buscarTrigramas = nltk.collocations.TrigramCollocationFinder.from_words(comentarios_tokens)

### Estratégia 1 - buscando bigramas e trigramas mais relevantes por frequência

In [19]:
# obtendo frequência dos bigramas
bigrama_freq = buscarBigramas.ngram_fd.items()
# criando dataframe com bigramas e suas frequências
FreqTabBigramas = pd.DataFrame(list(bigrama_freq),
                              columns = ["Bigrama","Freq"]).sort_values(by="Freq",ascending = False).reset_index(drop=True)
# obtendo frequência dos trigramas
trigrama_freq = buscarTrigramas.ngram_fd.items()
# criando dataframe com trigramas e suas frequências
FreqTabTrigramas = pd.DataFrame(list(trigrama_freq),
                              columns = ["Trigrama","Freq"]).sort_values(by="Freq",ascending = False).reset_index(drop=True)

In [20]:
FreqTabBigramas.head(5)

,Bigrama,Freq
0,"( , the)",8734
1,"( , i)",3662
2,"( , we)",3412
3,"(the, room)",3155
4,"(room, be)",3052


In [21]:
FreqTabTrigramas.head()

,Trigrama,Freq
0,"(the, room, be)",1639
1,"( , the, room)",1379
2,"( , it, be)",1178
3,"(the, staff, be)",1088
4,"( , the, staff)",866


In [22]:
# bigramas e trigramas ainda com muitas stopwords
# vamos filtra-los por tipo de cada palavra
en_stopwords = set(stopwords.words('english'))
def filtrar_tipo_token_bigrama(ngram):
    # verificar se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    # verificar se uma ou mais palavras é uma stopword ou um espaço em branco
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
    # tipos de token aceitáveis - adjetivos e substantivos
    acceptable_types = ('JJ','JJR','JJS','NN','NNS','NNP','NNPS')
    # subtipos
    second_type = ('NN','NNS','NNP','NNPS')
    # tag
    tags = nltk.pos_tag(ngram)
    # verificando se o bigrama segue o formato desejado
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False
def filtrar_tipo_token_trigrama(ngram):
    # verificar se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    # verificar se uma ou mais palavras é uma stopword ou um espaço em branco
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
    # tipos de token aceitáveis - adjetivos e substantivos
    acceptable_types = ('JJ','JJR','JJS','NN','NNS','NNP','NNPS')
    # subtipos
    second_type = ('JJ','JJR','JJS','NN','NNS','NNP','NNPS')
    # tag
    tags = nltk.pos_tag(ngram)
    # verificando se o trigrama segue o formato desejado
    if tags[0][1] in acceptable_types and tags[2][1] in second_type:
        return True
    else:
        return False

In [ ]:
# filtrar bigramas
bigramas_filtrados = FreqTabBigramas[FreqTabBigramas.Bigrama.map(lambda x: filtrar_tipo_token_bigrama(x))].reset_index(drop=True)
# filtrar trigramas
trigramas_filtrados = FreqTabTrigramas[FreqTabTrigramas.Trigrama.map(lambda x: filtrar_tipo_token_trigrama(x))].reset_index(drop=True)

In [ ]:
# bigramas mais relevantes por frequência
bigramas_filtrados.head(5)

In [ ]:
# trigramas mais relevantes por frequência
trigramas_filtrados.head(5)

### Estratégia 2 - relevância nos comentários por PMI  (Pointwise Mutual Information)

In [ ]:
# somente os ngramas com 20 ou mais ocorrências
buscarBigramas.apply_freq_filter(20)
buscarTrigramas.apply_freq_filter(20)

In [ ]:
# criando tabela de bigramas e trigramas com sua pontuação PMI
PMITabBigramas = pd.DataFrame(list(buscarBigramas.score_ngrams(bigramas.pmi)),
                             columns = ["Bigrama","PMI"]).sort_values(by="PMI",ascending=False)
PMITabTrigramas = pd.DataFrame(list(buscarTrigramas.score_ngrams(trigramas.pmi)),
                             columns = ["Trigrama","PMI"]).sort_values(by="PMI",ascending=False)

In [ ]:
# bigramas mais relevantes por PMI
PMITabBigramas.head(5)

In [ ]:
# trigramas mais relevantes por PMI
PMITabTrigramas.head(5)

### Estratégia 3 - Teste-t

In [ ]:
# criando tabela de bigramas e trigramas com sua pontuação no Teste-t
TestetTabBigramas = pd.DataFrame(list(buscarBigramas.score_ngrams(bigramas.student_t)),
                             columns = ["Bigrama","Teste-t"]).sort_values(by="Teste-t",ascending=False)
TestetTabTrigramas = pd.DataFrame(list(buscarTrigramas.score_ngrams(trigramas.student_t)),
                             columns = ["Trigrama","Teste-t"]).sort_values(by="Teste-t",ascending=False)

bigramas_t_filtrados = TestetTabBigramas[TestetTabBigramas.Bigrama.map(lambda x:filtrar_tipo_token_bigrama(x))]
trigramas_t_filtrados = TestetTabTrigramas[TestetTabTrigramas.Trigrama.map(lambda x:filtrar_tipo_token_trigrama(x))]

In [ ]:
# bigramas mais relevantes por Teste-t
bigramas_t_filtrados.head(5)

In [ ]:
# trigramas mais relevantes por Teste-t
trigramas_t_filtrados.head(5)

### Estratégia 4 - qui-quadrado

In [ ]:
# criando tabela de bigramas e trigramas com sua pontuação no qui-quadrado
TestetTabBigramas = pd.DataFrame(list(buscarBigramas.score_ngrams(bigramas.chi_sq)),
                             columns = ["Bigrama","Qui"]).sort_values(by="Qui",ascending=False)
TestetTabTrigramas = pd.DataFrame(list(buscarTrigramas.score_ngrams(trigramas.chi_sq)),
                             columns = ["Trigrama","Qui"]).sort_values(by="Qui",ascending=False)
bigramas_qui_filtrados = TestetTabBigramas[TestetTabBigramas.Bigrama.map(lambda x:filtrar_tipo_token_bigrama(x))]
trigramas_qui_filtrados = TestetTabTrigramas[TestetTabTrigramas.Trigrama.map(lambda x:filtrar_tipo_token_trigrama(x))]

In [ ]:
# bigramas mais relevantes por qui-quadrado
bigramas_qui_filtrados.head(5)

In [ ]:
# trigramas mais relevantes por qui-quadrado
trigramas_qui_filtrados.head(5)

### Comparando Estratégias

In [ ]:
# obtendo top10 bigramas e trigramas mais relevantes em cada estratégia 
metodo1_bigrama = bigramas_filtrados[:10].Bigrama.values
metodo2_bigrama = PMITabBigramas[:10].Bigrama.values
metodo3_bigrama = bigramas_t_filtrados[:10].Bigrama.values
metodo4_bigrama = bigramas_qui_filtrados[:10].Bigrama.values

metodo1_trigrama = trigramas_filtrados[:10].Trigrama.values
metodo2_trigrama = PMITabTrigramas[:10].Trigrama.values
metodo3_trigrama = trigramas_t_filtrados[:10].Trigrama.values
metodo4_trigrama = trigramas_qui_filtrados[:10].Trigrama.values

In [ ]:
# top 10 bigramas nas 4 estratégias
comparaBigramas = pd.DataFrame([metodo1_bigrama,metodo2_bigrama,metodo3_bigrama,metodo4_bigrama]).T
comparaBigramas.columns = ["Frequência","PMI","Teste-t","Teste Qui-quadrado"]
comparaBigramas

In [ ]:
# top 10 trigramas nas 4 estratégias
comparaTrigramas = pd.DataFrame([metodo1_trigrama,metodo2_trigrama,metodo3_trigrama,metodo4_trigrama]).T
comparaTrigramas.columns = ["Frequência","PMI","Teste-t","Teste Qui-quadrado"]
comparaTrigramas

### Conclusão:

Podemos ver que os métodos PMI e Qui-quadrado fornecem bons resultados. Seus resultados também são semelhantes. 
Mas os métodos de Frequência e Teste-t apresentam os melhores resutados e identicos.

Em aplicativos reais, podemos observar a lista e definir um limite em um valor a partir de quando a lista para de fazer sentido. Também podemos fazer testes diferentes para ver qual lista parece fazer mais sentido para um determinado conjunto de dados. 

Como alternativa, podemos combinar resultados de várias listas. Uma boa escolha é multiplicar o PMI e a Frequência para levar em consideração o aumento da probabilidade e a frequência da ocorrência. Ou multiplicar a frequência pelo Teste-t criando assim um índice único de relevância das Collocations.

Para este projeto, vamos considerar as Collocations calculadas por Frequência como as mais relevantes. A escolha se deve ao fato de que as suposições para o Teste-t não foram validadas e usar seu resultado seria inadequado.